A method that is called implicitly by Python to execute a certain operation on a type, such as addition. Such methods have names starting and ending with double underscores.

In [1]:
# When we do 1+2, it works like:
(1).__add__(2)

3

#### Controlling Object creation process

In [2]:
# init - used to initialize the object

class Demo:
    def __init__(self,name):
        self.name = name
    
    def disp(self):
        print(f"Name: {self.name}")

demo = Demo("satya")
demo.disp()

Name: satya


In [4]:
# new - used to initialize the class which is then passed to init

class Demo:
    def __new__(cls):
        print("Creating Demo instance")
        return super().__new__(cls)
    
    def hello(self):
        print("Hello")
    
demo = Demo()
demo.hello()

Creating Demo instance
Hello


#### Objects as strings

In [5]:
class Demo:
    def __init__(self,name):
        self.name = name
    
    def disp(self):
        print(f"Name: {self.name}")

demo = Demo("satya")
print(demo)
demo

In [7]:
# the above are printed and returned when object is printed or called. to change this is 

class Demo:
    def __init__(self,name):
        self.name = name
    
    def disp(self):
        print(f"Name: {self.name}")

    # Informal Representation
    def __str__(self) -> str:
        return "Demo printed"
    
    # Formal Representation
    def __repr__(self) -> str:
        return "Demo called"

demo = Demo("satya")
print(demo)
demo

Demo printed


Demo called